In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim import Adam
from sklearn.model_selection import train_test_split
from PIL import Image
import numpy as np
import cv2
import timm # 尽管EDSR不直接用timm，但保留以防其他用途

# ----------------------------------------
# 1. 设置和数据加载部分 (保持不变)
# ----------------------------------------
patches_folder = r"C:\Users\Alpaca_YT\pythonSet\lung_slices_dataset\lung_slice_xy"
output_model_dir = "Train_EDSR_Full_newlung" # 更换输出文件夹名称
os.makedirs(output_model_dir, exist_ok=True)
num_epochs = 15
batch_size = 8
learning_rate = 1e-4

all_fns = sorted([f for f in os.listdir(patches_folder) if f.lower().endswith(".jpg")])
all_indices = list(range(len(all_fns)))
train_idxs, val_idxs = train_test_split(all_indices, test_size=0.25, random_state=42)

class RotLowHighDataset(Dataset):
    def __init__(self, patches_folder, indices, all_fns_list, transform=None):
        super().__init__()
        self.patches_folder = patches_folder
        self.transform = transform or transforms.ToTensor()
        self.fns = [all_fns_list[i] for i in indices]
    def __len__(self): return len(self.fns) * 2
    def __getitem__(self, idx):
        img_idx, rot_flag = idx // 2, idx % 2
        fn = self.fns[img_idx]
        img_path = os.path.join(self.patches_folder, fn)
        arr = np.array(Image.open(img_path).convert("L"))
        if rot_flag == 1: arr = np.rot90(arr, k=1)
        down_arr = cv2.resize(arr, (256, 32), interpolation=cv2.INTER_AREA)
        up_img = cv2.resize(down_arr, (256, 256), interpolation=cv2.INTER_LINEAR)
        inp_t = self.transform(Image.fromarray(up_img))
        tgt_t = self.transform(Image.fromarray(arr))
        return inp_t, tgt_t

class PlainLowHighDataset(Dataset):
    def __init__(self, patches_folder, indices, all_fns_list, transform=None):
        super().__init__()
        self.patches_folder = patches_folder
        self.transform = transform or transforms.ToTensor()
        self.fns = [all_fns_list[i] for i in indices]
    def __len__(self): return len(self.fns)
    def __getitem__(self, idx):
        fn = self.fns[idx]
        img_path = os.path.join(self.patches_folder, fn)
        arr = np.array(Image.open(img_path).convert("L"))
        down_arr = cv2.resize(arr, (256, 32), interpolation=cv2.INTER_AREA)
        up_img = cv2.resize(down_arr, (256, 256), interpolation=cv2.INTER_LINEAR)
        inp_t = self.transform(Image.fromarray(up_img))
        tgt_t = self.transform(Image.fromarray(arr))
        return inp_t, tgt_t

# ----------------------------------------
# 2. 定义 EDSR 模型 (完全替换 SwinUnet 部分)
# ----------------------------------------

class ResidualBlock(nn.Module):
    """EDSR的残差块，不包含BN层"""
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)

    def forward(self, x):
        residual = self.conv1(x)
        residual = self.relu(residual)
        residual = self.conv2(residual)
        return x + residual # 残差连接

class Upsampler(nn.Module):
    """EDSR的上采样模块，使用PixelShuffle"""
    def __init__(self, in_channels, scale_factor):
        super(Upsampler, self).__init__()
        self.conv = nn.Conv2d(in_channels, in_channels * (scale_factor ** 2), kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(scale_factor)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.relu(self.pixel_shuffle(self.conv(x)))

class EDSR(nn.Module):
    def __init__(self, in_channels=1, out_channels=1, scale_factor=8, num_res_blocks=16, feature_channels=64):
        super(EDSR, self).__init__()
        
        self.scale_factor = scale_factor # 放大倍数，这里是8 (32->256)

        # 初始特征提取
        self.head = nn.Conv2d(in_channels, feature_channels, kernel_size=3, padding=1)

        # 残差块主体
        body = [ResidualBlock(feature_channels) for _ in range(num_res_blocks)]
        self.body = nn.Sequential(*body)
        
        # 最后的卷积层 (在残差块之后，上采样之前)
        self.conv_after_res = nn.Conv2d(feature_channels, feature_channels, kernel_size=3, padding=1)

        # 上采样部分 (实现 8x 放大)
        # 256x32 输入，目标 256x256。实际是从32高放大到256高，即 8x
        # 我们可以通过多次2x PixelShuffle实现
        
        # 假设输入是 256x32 (低分辨率输入)
        # EDSR的典型实现是在特征空间进行上采样
        # 我们的输入是已经插值到 256x256 的，但实际上是从 256x32 来的
        # 因此，网络是学习从“插值后的LR”到“HR”的映射

        # 这里我们需要思考 scale_factor=8 的应用场景
        # 您的数据加载是：
        #   1. 原始 HR (256x256) -> 下采样到 LR (256x32)
        #   2. LR (256x32) -> 上采样到 LR_bicubic (256x256) 作为网络输入
        # 所以，EDSR的任务是从 LR_bicubic (256x256) 学习到 HR (256x256)
        # 在这种情况下，我们不需要在网络内部进行显式的 scale_factor=8 的上采样
        # EDSR的主体是在一个固定的分辨率上运行，然后最终输出。
        # 这里我们将EDSR设计为接收 256x256 伪LR输入，输出 256x256 HR。

        # 原始EDSR通常会将 LR 输入直接放大到 HR
        # 鉴于您的数据集处理方式，EDSR的“上采样”部分可以简化
        # 或者，我们可以让EDSR接收 256x32，然后内部进行 8x 上采样
        # 鉴于您提供的数据预处理，我们让EDSR直接处理 256x256 -> 256x256
        # 这意味着模型学的是一个图像到图像的映射，而不是低分辨率到高分辨率的传统超分

        # -------------------------------------------------------------
        # 重新考虑EDSR的上采样部分：
        # 如果模型直接接收 256x256 的插值后的输入，并输出 256x256 的HR
        # 那么EDSR的“上采样”模块就可能不是必需的，或者只需要一个最终的重建卷积。
        # 原始EDSR是在低分辨率特征图上进行上采样，然后输出高分辨率图像。
        # 您的输入 `inp_t` 已经是 `256x256`。
        # 故，EDSR的结构应该是：
        # 特征提取 -> 多个残差块 -> 最后的卷积层 -> 输出。

        # 这里的 EDSR 不进行内部的上采样操作，因为它接收的输入已经是目标尺寸。
        # 它是一个图像到图像的映射网络，目标是消除插值引入的伪影，并恢复细节。
        # 这更像是一个图像去噪或增强任务。
        # -------------------------------------------------------------

        # 最终重建层
        self.tail = nn.Conv2d(feature_channels, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        # EDSR通常会将输入减去平均值，这里我们假设输入已在0-1范围
        # 或者在实际训练时进行数据归一化
        
        # 初始特征提取
        x = self.head(x)
        
        # 残差块主体
        res = self.body(x)
        res = self.conv_after_res(res) # 残差块后的卷积
        x = x + res # 全局残差连接 (跳过主体部分)

        # 最终重建
        x = self.tail(x)
        return x

# ----------------------------------------
# 3. 准备训练
# ----------------------------------------
transform = transforms.ToTensor()
train_dataset = RotLowHighDataset(patches_folder, train_idxs, all_fns, transform)
val_dataset = PlainLowHighDataset(patches_folder, val_idxs, all_fns, transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 实例化 EDSR 模型 ---
# 注意：EDSR在这里被配置为接受 256x256 的输入并输出 256x256 的图像
# 因为您的数据加载器已经将 256x32 的 LR 图像双线性插值到了 256x256
# 因此，EDSR的任务是“优化”这个插值后的 256x256 图像到真正的 HR 256x256
model = EDSR(in_channels=1, out_channels=1, num_res_blocks=32, feature_channels=64).to(device) # 增加残差块数量以提升能力
# EDSR通常不使用预训练，所以这里没有pretrained参数

optimizer = Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss() # 基础的MSE损失

# --- 定义高频损失所需组件 ---
laplacian_kernel = torch.tensor(
    [[0.0, -1.0, 0.0],
     [-1.0, 4.0, -1.0],
     [0.0, -1.0, 0.0]],
    device=device, dtype=torch.float32
).view(1, 1, 3, 3)

lambda_hf = 0.5 # 高频损失的权重

def high_freq_loss(pred, target):
    """计算预测和目标之间高频分量的MSE损失"""
    pred_lap = F.conv2d(pred, laplacian_kernel, padding=1)
    tgt_lap  = F.conv2d(target, laplacian_kernel, padding=1)
    return F.mse_loss(pred_lap, tgt_lap)


# ----------------------------------------
# 4. 训练循环
# ----------------------------------------
print(f"开始在 {device} 上训练 EDSR (使用高频加权损失)...")
best_val_loss = float('inf') # 用于保存最佳模型
for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss = 0.0
    for inp, tgt in train_loader:
        inp, tgt = inp.to(device), tgt.to(device)
        optimizer.zero_grad()
        out = model(inp)
        
        mse_train = criterion(out, tgt)
        hf_train = high_freq_loss(out, tgt)
        loss = mse_train + lambda_hf * hf_train # 总损失 = MSE + λ * 高频损失
        
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inp.size(0)
    avg_train_loss = train_loss / len(train_dataset)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inp_v, tgt_v in val_loader:
            inp_v, tgt_v = inp_v.to(device), tgt_v.to(device)
            out_v = model(inp_v)
            
            mse_val = criterion(out_v, tgt_v)
            hf_val = high_freq_loss(out_v, tgt_v)
            loss_v = mse_val + lambda_hf * hf_val

            val_loss += loss_v.item() * inp_v.size(0)
    avg_val_loss = val_loss / len(val_dataset)

    print(f"Epoch {epoch:02d}/{num_epochs} | Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f}")

    # 保存最佳模型
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        ckpt_path = os.path.join(output_model_dir, f"EDSR_Full_best.pth") # 保存最佳模型
        torch.save(model.state_dict(), ckpt_path)
        print(f"  --> Saving best model at epoch {epoch} with Val Loss: {best_val_loss:.6f}")

    # 也可以选择每个epoch都保存，但为了简洁和效率，这里只保存最佳
    # ckpt_path_epoch = os.path.join(output_model_dir, f"EDSR_Full_epoch{epoch:02d}.pth")
    # torch.save(model.state_dict(), ckpt_path_epoch)

print(f"\n训练完毕，最佳模型已保存在 '{output_model_dir}/'")

C:\Users\Alpaca_YT\anaconda3\envs\mphy0041-cw2-pt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


开始在 cuda 上训练 EDSR (使用高频加权损失)...
Epoch 01/15 | Train Loss: 0.020976 | Val Loss: 0.014340
  --> Saving best model at epoch 1 with Val Loss: 0.014340
